<a href="https://colab.research.google.com/github/IvanLudvig/eggcounting/blob/master/eggcounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from urllib.request import urlopen

In [2]:
with urlopen('https://raw.githubusercontent.com/IvanLudvig/eggcounting/master/data/data1.json') as f:
    data = json.load(f)['messages']
    df = pd.DataFrame(data)

In [3]:
df

,id,type,timestamp,timestampEdited,callEndedTimestamp,isPinned,content,author,attachments,embeds,reactions,mentions,reference
0,886460807087550536,Default,2021-09-12T03:58:46.055+00:00,None,None,False,1,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
1,886460808425537597,Default,2021-09-12T03:58:46.374+00:00,None,None,False,2,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
2,886460810455580732,Default,2021-09-12T03:58:46.858+00:00,None,None,False,3,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
3,886460811722227713,Default,2021-09-12T03:58:47.16+00:00,None,None,False,4,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
4,886460813790023730,Default,2021-09-12T03:58:47.653+00:00,None,None,False,5,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102308,897490952921432084,Default,2021-10-12T14:28:37.752+00:00,None,None,False,102246,"{'id': '472032231401979913', 'name': 'brainlet...",[],[],[],[],NaN
102309,897490954813050880,Default,2021-10-12T14:28:38.203+00:00,None,None,False,102247,"{'id': '472032231401979913', 'name': 'brainlet...",[],[],[],[],NaN
102310,897490958873133086,Default,2021-10-12T14:28:39.171+00:00,None,None,False,102248,"{'id': '472032231401979913', 'name': 'brainlet...",[],[],[],[],NaN
102311,897490993539076136,Default,2021-10-12T14:28:47.436+00:00,None,None,False,102249,"{'id': '472032231401979913', 'name': 'brainlet...",[],[],[],[],NaN


In [4]:
df['user'] = df['author'].map(lambda x: x['name'])

In [5]:
df = df[['id', 'timestamp', 'content', 'user']]

filter only valid counts

In [6]:
df = df[df['content'].str.isnumeric()]

# Total counts

In [7]:
counts_df = df['user'].value_counts(sort=True).reset_index()
counts_df.columns = ['user', 'counts']

In [8]:
counts_df.index += 1
counts_df

,user,counts
1,Whit4You,38544
2,brainlet 🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚,36698
3,thelegendary08,6341
4,const,3892
5,Pranoodles🍜 (she/her),3804
6,ob69jl,1715
7,country croocks,1610
8,striker,1571
9,FortniteDude70,771
10,Dawidluna,718


# 1K days

In [9]:
df.head()

,id,timestamp,content,user
0,886460807087550536,2021-09-12T03:58:46.055+00:00,1,thelegendary08
1,886460808425537597,2021-09-12T03:58:46.374+00:00,2,thelegendary08
2,886460810455580732,2021-09-12T03:58:46.858+00:00,3,thelegendary08
3,886460811722227713,2021-09-12T03:58:47.16+00:00,4,thelegendary08
4,886460813790023730,2021-09-12T03:58:47.653+00:00,5,thelegendary08


In [10]:
df['date'] = df['timestamp'].apply(lambda ts: ts.split('T')[0])

In [11]:
days_df = df[['user', 'date']]
days_df.head()

,user,date
0,thelegendary08,2021-09-12
1,thelegendary08,2021-09-12
2,thelegendary08,2021-09-12
3,thelegendary08,2021-09-12
4,thelegendary08,2021-09-12


In [12]:
group = days_df.groupby('user')
group = group.apply(lambda x: x['date'].to_numpy()).reset_index()
group.columns = ['user', 'dates']
group.head()

,user,dates
0,0_0,"[2021-09-12, 2021-09-12, 2021-09-12, 2021-09-1..."
1,Brendan7625,"[2021-09-18, 2021-09-18, 2021-09-18, 2021-09-1..."
2,Dawidluna,"[2021-09-16, 2021-09-16, 2021-09-16, 2021-09-1..."
3,DrToad866 🅱,"[2021-10-03, 2021-10-03, 2021-10-03, 2021-10-0..."
4,FortniteDude70,"[2021-09-12, 2021-09-12, 2021-09-12, 2021-09-1..."


In [13]:
for date in days_df['date'].unique():
    group[date] = group['dates'].map(lambda x: (x == date).sum())

In [14]:
group = group.drop(columns=['dates'])
group.head()

,user,2021-09-12,2021-09-13,2021-09-14,2021-09-15,2021-09-16,2021-09-17,2021-09-18,2021-09-19,2021-09-20,2021-09-21,2021-09-22,2021-09-23,2021-09-24,2021-09-25,2021-09-26,2021-09-27,2021-09-28,2021-09-29,2021-09-30,2021-10-01,2021-10-02,2021-10-03,2021-10-04,2021-10-05,2021-10-06,2021-10-07,2021-10-08,2021-10-09,2021-10-10,2021-10-11,2021-10-12
0,0_0,5,47,0,0,0,0,0,42,8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0
1,Brendan7625,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dawidluna,0,0,0,0,200,80,0,0,0,32,0,0,0,0,0,0,0,0,120,263,0,0,15,0,0,0,8,0,0,0,0
3,DrToad866 🅱,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,0,0,27,0,0,0,0,0,0
4,FortniteDude70,115,180,0,0,0,0,0,0,0,0,0,0,0,204,124,5,10,10,10,0,20,11,0,0,0,0,0,37,27,6,12


In [15]:
counts = group.drop(columns=['user']).values

In [16]:
group['counts_list'] = counts.tolist()
group = group[['user', 'counts_list']]
group.head()

,user,counts_list
0,0_0,"[5, 47, 0, 0, 0, 0, 0, 42, 8, 0, 0, 0, 0, 0, 1..."
1,Brendan7625,"[0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0..."
2,Dawidluna,"[0, 0, 0, 0, 200, 80, 0, 0, 0, 32, 0, 0, 0, 0,..."
3,DrToad866 🅱,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,FortniteDude70,"[115, 180, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20..."


### Total 1k days

In [17]:
group['1k_days'] = group['counts_list'].map(lambda x: (np.array(x) >= 1000).sum())
total_1kdays = group[['user', '1k_days']].sort_values(by='1k_days', ascending=False).reset_index(drop=True)
total_1kdays = total_1kdays[total_1kdays['1k_days']>0]

In [18]:
total_1kdays.index += 1
total_1kdays

,user,1k_days
1,Whit4You,14
2,brainlet 🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚,13
3,thelegendary08,1
4,ob69jl,1
5,const,1


## Max 1k streak

In [19]:
def get_max_streak(counts_list):
    bool_array = np.array(counts_list) >= 1000
    bool_array = np.split(bool_array, np.where(np.diff(bool_array) != 0)[0]+1)
    bool_array = list(filter(lambda x: x[0], bool_array))
    bool_array = [x.shape[0] for x in bool_array]
    return max(bool_array) if len(bool_array) > 0 else 0

In [20]:
group['max_streak'] = group['counts_list'].map(get_max_streak)

In [21]:
max_streak = group[['user', 'max_streak']].sort_values(by='max_streak', ascending=False).reset_index(drop=True)
max_streak = max_streak[max_streak['max_streak']>0]

In [22]:
max_streak.index += 1
max_streak

,user,max_streak
1,brainlet 🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚🥚,6
2,Whit4You,5
3,thelegendary08,1
4,ob69jl,1
5,const,1
